# Synthesis plot of existing observations

In [ ]:
import sys
import numpy as np
import xarray as xr
import pandas as pd
import cmocean
import matplotlib.pyplot as plt

sys.path.append('..')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}

from forcing import Forcing
from ideal_geometry import IdealGeometry

In [ ]:
# various published melt estimates

# satellite derived
obs1 = {'PineIsland'   :{'Rignot13'         :([2003,2009],16.2),
                         'Adusumilli20'     :([1994,2019],14. ),
                         'Gourmelen17'      :([2008,2017], 5.5),
                         'Shean19'          :([2008,2016],13.9),
                        },
        'Thwaites'     :{'Rignot13'         :([2003,2009],11.9),
                         'Adusumilli20'     :([1994,2019],27. ),
                        },
        'CrossonDotson':{'Rignot13'         :([2003,2009],9.3),
                         'Adusumilli20'     :([1994,2019],6.2),
                         'Goldberg19'       :([2010,2015],[6.86,5.58]),
                        },
        'Crosson'      :{'Rignot13'         :([2003,2009],11.9),
                         'Adusumilli20'     :([1994,2019], 7.8),
                         'Goldberg19'       :([2010,2015],[7.15,5.39]),
                        },
        'Dotson'       :{'Rignot13'         :([2003,2009],7.8),
                         'Adusumilli20'     :([1994,2019],5.4),
                         'Gourmelen17'      :([2010,2017],5.5),
                         'Jenkins18'        :([2012,2017],3.5),
                         'Goldberg19'       :([2010,2015],[6.68,5.70]),
                        }
       }

# hydrography
obs2 = {'PineIsland'   :{'Dutrieux14'       :[(2007,17.1), (2012,5.6)],
                        },
       'Dotson'       :{'Jenkins18'         :(2009,18),
                        'Randall-Goodwin15' :(2011,15),
                        'Adusumilli20'      :(2009,12),
                        }
       }

colors = {'Rignot13'         : 'C0',
          'Adusumilli20'     : 'C1',
          'Gourmelen17'      : 'C2',
          'Shean19'          : 'C3',
          'Goldberg19'       : 'C7',
          'Jenkins18'        : 'C4',
          'Dutrieux14'       : 'C5',
          'Randall-Goodwin15': 'C6',
         }

target = {'PineIsland':{'mean':[13,15], 'cold':8, 'warm':17},
          'Dotson'    :{'mean':[5.5,7], 'cold':4, 'warm':11},
         }

In [ ]:
# digitized potential temperature profiles
dfJ = pd.read_csv('../../data/hydrography/Jenkins18.csv')
dfD = pd.read_csv('../../data/hydrography/Dutrieux14.csv')

In [ ]:
def make_DataArray(df):
    x_ = np.arange(-1000,0.01,5)  # new depth coordinate
    n_profiles = int(len(df.columns)/2)
    da = xr.DataArray(dims=('time','depth'),
                      coords={'depth':x_, 'time':df.columns[::2].astype(int)}
                     )
    for i in range(n_profiles):
        yr = int(df.columns[2*i])

        x = df.iloc[1:,2*i].values.astype(float)
        y = df.iloc[1:,2*i+1].values.astype(float)
        x = x[y.argsort()]
        y = y[y.argsort()]

        da[i,:] = np.interp(x_,y,x)
    return da.sortby('time')
    
dD = make_DataArray(dfD)
dJ = make_DataArray(dfJ)


In [ ]:
def year_color(year):
    ys,ye = 1994,2016
    return plt.get_cmap('tab20c_r')(((year-ys)/(ye-ys))**1.5)

In [ ]:
def pub_marker(pub):
    if pub in ['Rignot13','Shean19','Gourmelen17','Adusumilli20','Goldberg19']:
        marker = None
    elif pub in ['Dutrieux14','Jenkins18','Randall-Goodwin15']:
        marker = 's'
    else:
        raise ValueError(f'publication {pub} not defined for marker')
    return marker

In [ ]:
f = plt.figure(figsize=(8,8))

""" melt estimates """
axt = f.add_axes([.1,.77,.7,.22])
axb = f.add_axes([.1,.53 ,.7,.22])
axb.set_xlabel('time [year C.E.]')

for i, geom in enumerate(obs1.keys()):
    if geom in ['CrossonDotson', 'Dotson', 'Crosson']:
        ax = axb
        if geom=='CrossonDotson':  ls = '-'
        elif geom=='Dotson':       ls = '--'
        elif geom=='Crosson':      ls = ':'
    elif geom=='PineIsland':  ax, ls = axt, '-'
    else: continue
    ls = ['-','','-','--',':'][i]
    for j, pub in enumerate(obs1[geom].keys()):
        m = pub_marker(pub)
        x = obs1[geom][pub]
        print(x[0], x[1], colors[pub], ls)
        if type(x[1])==float:
            ax.plot(x[0],2*[x[1]], c=colors[pub], ls=ls, marker=m)
        elif type(x[1])==list:
            for value in x[1]:
                ax.plot(x[0],2*[value], c=colors[pub], ls=ls, marker=m)
        
for i, geom in enumerate(obs2.keys()):
    if geom in ['CrossonDotson', 'Dotson', 'Crosson']:   ax, ls = axb, '--'
    elif geom=='PineIsland':  ax, ls = axt, '-'
    else: continue
    for j, pub in enumerate(obs2[geom].keys()):
        m = pub_marker(pub)
        x = obs2[geom][pub]
        if type(x)==list:
            for est in x:
                ax.plot([est[0],est[0]+1],2*[est[1]], ls=ls, color=colors[pub], marker=m)
        elif type(x)==tuple:
            ax.plot([x[0],x[0]+1],2*[x[1]], ls=ls, color=colors[pub], marker=m)
    
for i, geom in enumerate(['Crosson-Dotson', 'Dotson', 'Crosson']):
    axb.plot([],[], ls=['-','--',':'][i], c='grey', label=geom)
axb.legend(ncol=3, loc='lower left', frameon=False)

for i, pub in enumerate(colors.keys()):
#     if i==4:  axt.plot([],[],c=None, label=' ', alpha=0)
    axt.plot([],[],c=colors[pub], label=pub, marker=m)
axt.legend(ncol=2, loc='lower left', frameon=False)

for i, ax in enumerate([axt, axb]):
    ax.set_xlim((1993.5,2019.5))
    ax.set_ylim((0,None))
    ax.text(.01,.88, ['a) Pine Island','c) Crosson-Dotson'][i], fontsize=12, transform=ax.transAxes)

f.text(.015,.75, 'melt rate estimate  [m/yr]', rotation=90, ha='center', va='center')
f.text(.015,.25, 'depth  [m]', rotation=90, ha='center', va='center')

""" validation/tuning """
ax1 = f.add_axes([.83,.77,.16,.22], sharey=axt)
ax2 = f.add_axes([.83,.53,.16,.22], sharey=axb, sharex=ax1)
plt.setp(ax1.get_xticklabels(), visible=False)
ax1.set_xticklabels([])
ax2.set_xticks(np.arange(6))
ax2.set_xticklabels(['target',r'M$_+$','Plume','PICO','PICOP','Layer'], rotation=45)
ax2.set_xlim((-1,5.5))
for i, geom in enumerate(['PineIsland', 'Dotson']):
    ax = [ax1,ax2][i]
    ax.axhspan(*target[geom]['mean'], color='lightgrey')
    ax.scatter(0, np.mean(target[geom]['mean']), marker='D', color='k', zorder=2)
    ax.scatter(0, target[geom]['cold'], marker='^', color='k')
    ax.scatter(0, target[geom]['warm'], marker='v', color='k')
    ax.text(.04,.88, ['b)','d)'][i], fontsize=12, transform=ax.transAxes)
    plt.setp(ax.get_yticklabels(), visible=False)

""" hydrography """
axl = f.add_axes([.1,.07,.37,.35])
axr = f.add_axes([.49,.07,.37,.35])

years = []
for i, da in enumerate([dJ,dD]):
    ax = [axr,axl][i]
    if i==0:
        Tdeep, ztclw, ztclc = 0.7, -400, -700
    elif i==1:
        Tdeep, ztclw, ztclc = 1.2, -400, -600
    dc = Forcing(IdealGeometry('Ocean1').create()).tanh2(ztcl=ztclc, Tdeep=Tdeep)
    dw = Forcing(IdealGeometry('Ocean1').create()).tanh2(ztcl=ztclw, Tdeep=Tdeep)
    ax.set_title(['f) Dotson (Jenkins et al. \'18)','e) Pine Island (Dutrieux et al. \'14)'][i], loc='left')
    ax.set_xlabel(r'potential temperature  [$^\circ$C]')
    for y in da.time:
        ax.plot(da.sel(time=y), da.depth, c=year_color(y), label=y.values)
        if y.values not in years:   years.append(int(y.values))
    c, = ax.plot(dc.Tz, dc.z, c='b', label='Cold')
    w, = ax.plot(dw.Tz, dw.z, c='r', label='Warm')
    ax.set_ylim((-1000,0))

axr.set_yticklabels([])
axt.set_xticklabels([])

years.sort()
handles = []
for i, y in enumerate(years):
    l, = axr.plot([],[],c=year_color(y), label=y)
    handles.append(l)
axr.legend(handles=handles+[w,c], bbox_to_anchor=(1.01, 0.5), loc='center left')

plt.savefig('../../figures/observations.eps')

In [ ]:
years

In [ ]:
import matplotlib.colors as mcolors

colors1 = plt.cm.RdYlBu_r(np.linspace(0., 1, 4))
colors2 = plt.cm.RdYlGn(np.linspace(0, 1, 6))

# combine them and build a new colormap
colors3 = np.vstack((colors1, colors2))
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors3)


data = np.random.rand(10,10) * 2 - 1
plt.pcolor(data, cmap=mymap)
plt.colorbar()
plt.show()

In [ ]:
years

In [ ]:
def year_color2(year):
    ys,ye = 1994,2016
    return mymap(((year-ys)/(ye-ys))**1.1)

plt.axvspan(2006,2010, color='red', alpha=.1)
for i, y in enumerate(years):
    plt.scatter(y, y, color=mymap(i/10))

### individual figures

In [ ]:
f = plt.figure(figsize=(10,5))
for i, geom in enumerate(obs1.keys()):
    c = ['C0','C2','C3','C1','C4'][i] 
    plt.plot([],[], c=c, label=geom)
    for j, pub in enumerate(obs1[geom].keys()):
        ls = ['-','--','-.',':',''][j]
        x = obs1[geom][pub]
        print(x)
        plt.plot(x[0],2*[x[1]], c=c, ls=ls, label=pub)
        
for i, geom in enumerate(obs2.keys()):
    c = ['C0','C4'][i] 
    for j, pub in enumerate(obs2[geom].keys()):
        x = obs2[geom][pub]
        print(x)
        if type(x)==list:
            for est in x:
                plt.scatter(est[0],est[1], color=c, marker='o', label=pub)
        elif type(x)==tuple:
            plt.scatter(x[0],x[1], color=c, marker=['+','>','<'][j], label=pub)
plt.xlabel('time [year C.E.]')
plt.ylabel('melt rate estimate [m/yr]')

# hydrographic observations
PIG = [-6,0,7,9,10,12]
D = [0,6,7,9,11,12,14,16]
plt.scatter([x+2000 for x in PIG], len(PIG)*[1.], marker='x', color='C0', label='hydrogr. obs. Dutrieux')
plt.scatter([x+2000 for x in D]  , len(D)*[0.5] , marker='x', color='C4', label='hydrogr. obs. Jenkins??')

plt.legend(ncol=4, loc='upper center')
plt.ylim((None, 50))

> separate for Pine Island and Crosson-Dotson

- Rignot estimates (03-08) systematically higher than Adusumilli estimates (94-18).

In [ ]:
f, ax = plt.subplots(1,2, figsize=(12,3), sharey=True)
dD.T.plot(ax=ax[0], add_colorbar=False)
dJ.T.plot(ax=ax[1])

In [ ]:
def year_color(year):
    ys,ye = 1994,2016
    return plt.get_cmap('tab20c_r')(((year-ys)/(ye-ys))**1.5)

f, ax = plt.subplots(1,2, figsize=(12,5), sharey=True)
for i, da in enumerate([dJ,dD]):
    if i==0:
        Tdeep, ztclw, ztclc = 0.7, -400, -700
    elif i==1:
        Tdeep, ztclw, ztclc = 1.2, -400, -600
    dc = Forcing(IdealGeometry('Ocean1').create()).tanh2(ztcl=ztclc, Tdeep=Tdeep)
    dw = Forcing(IdealGeometry('Ocean1').create()).tanh2(ztcl=ztclw, Tdeep=Tdeep)
    ax[i].set_title(['Dotson (Jenkins \'14)','PIG (Dutrieux \'18)'][i])
    ax[i].set_xlabel(r'potential temperature [$^\circ$C]')
    for y in da.time:
        ax[i].plot(da.sel(time=y), da.depth, c=year_color(y), label=y.values)
    ax[i].plot(dc.Tz, dc.z, c='b', label='Cold')
    ax[i].plot(dw.Tz, dw.z, c='r', label='Warm')
    ax[i].set_ylim((-1000,0))
    ax[i].legend(ncol=2)